In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as dsets

In [2]:
import visdom
vis = visdom.Visdom()

Setting up a new session...


In [3]:
vis.text('hello world', env='main')


'window_3a43881b9523e0'

In [4]:
a = torch.randn(3,200,200)
vis.image(a)

'window_3a43884359d6e2'

In [5]:
vis.images(torch.randn(3,3,28,28))

'window_3a43885b267bea'

In [6]:
mnist = dsets.MNIST(root='../Data/MNIST_data', train=True, transform=torchvision.transforms.ToTensor(), download=True)
cifar10 = dsets.CIFAR10(root='../Data/cifar10', train=True, transform=torchvision.transforms.ToTensor(), download=True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ../cifar10/cifar-10-python.tar.gz to ../cifar10


In [8]:
data = cifar10.__getitem__(0)
print(data[0].shape)
vis.images(data[0], env='main')

torch.Size([3, 32, 32])


'window_3a439bb251e85e'

In [9]:
data = mnist.__getitem__(0)
print(data[0].shape)
vis.images(data[0], env='main')

torch.Size([1, 28, 28])


'window_3a43b15673f73c'

In [10]:
data_loader = torch.utils.data.DataLoader(dataset=mnist, batch_size=32, shuffle=False)
for num, value in enumerate(data_loader):
    value = value[0]
    print(value.shape)
    vis.images(value)
    break

torch.Size([32, 1, 28, 28])


In [27]:
vis.close(env='main')

''

In [16]:
y_data = torch.randn(5)
plt = vis.line(Y=y_data)

In [17]:
x_data = torch.Tensor([1,2,3,4,5])
plt = vis.line(X=x_data, Y=y_data)


In [20]:
y_append = torch.randn(1)
x_append = torch.Tensor([6])

vis.line(Y=y_append, X=x_append, win=plt, update='append')

'window_3a43b1dbf1b864'

In [22]:
num = torch.Tensor(list(range(0,10)))
num = num.view(-1,1)
num = torch.cat((num,num),dim=1)

plt = vis.line(Y=torch.randn(10,2), X = num)

In [23]:
plt = vis.line(Y=y_data, X=x_data, opts=dict(title='Test', showlegend=True))

In [24]:
plt = vis.line(Y=y_data, X=x_data, opts=dict(title='Test', legend=['1번'], showlegend=True))

In [30]:
vis.close(env='main')

''

In [34]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

import torch.nn.init

def loss_tracker(loss_plot, loss_value, num):
    vis.line(X=num, Y=loss_value, win=loss_plot, update='append')



In [31]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [32]:
learning_rate = 1e-3
training_epochs = 10
batch_size = 50

In [36]:
mnist_train = dsets.MNIST(root='../Data/MNIST_data/',
                            train = True,
                            transform=transforms.ToTensor(),
                            download=True)
mnist_test = dsets.MNIST(root='../Data/MNIST_data/',
                            train = False,
                            transform=transforms.ToTensor(),
                            download=True)

In [37]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                            batch_size = batch_size,
                                            shuffle = True,
                                            drop_last = True)
# light model because of training without gpu
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 5, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(5, 10, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.fc = nn.Linear(7*7*10, 10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

                                           

In [40]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr = learning_rate)
loss_plt = vis.line(Y=torch.Tensor(1).zero_(), opts=dict(title='loss_tracker', legend=['loss'], showlegend=True))

total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0
    for x, y in data_loader:
        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()
        hypothesis = model(x)

        cost = criterion(hypothesis, y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch
    print(f'epoch = {epoch}, cost = {avg_cost}')
    loss_tracker(loss_plt, torch.Tensor([avg_cost]),torch.Tensor([epoch]))

epoch = 0, cost = 0.3853902220726013
epoch = 1, cost = 0.12031114846467972
epoch = 2, cost = 0.08983518928289413
epoch = 3, cost = 0.07280782610177994
epoch = 4, cost = 0.06554210186004639
epoch = 5, cost = 0.05909610539674759
epoch = 6, cost = 0.05282540246844292
epoch = 7, cost = 0.049169011414051056
epoch = 8, cost = 0.045402053743600845
epoch = 9, cost = 0.042625315487384796


Connection to remote host was lost.
on_close() takes 1 positional argument but 3 were given
[Errno 111] Connection refused
on_close() takes 1 positional argument but 3 were given
[Errno 111] Connection refused
on_close() takes 1 positional argument but 3 were given
[Errno 111] Connection refused
on_close() takes 1 positional argument but 3 were given
[Errno 111] Connection refused
on_close() takes 1 positional argument but 3 were given
[Errno 111] Connection refused
on_close() takes 1 positional argument but 3 were given
[Errno 111] Connection refused
on_close() takes 1 positional argument but 3 were given
[Errno 111] Connection refused
on_close() takes 1 positional argument but 3 were given
